In [1]:
from latex2sympy2 import latex2sympy
from pylatexenc.latex2text import LatexNodes2Text
import sympy as sp
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
def parse_assigment_with_dots(expression):
    # Expresiones equivalentes para las expresiones con \cdots o \ldots
    equivalents = {
        r"N = 9 + 99 + 999 + 9999 + \cdots + \underbrace{99\ldots 99}_\text{321 digits}.": r"N = \sum_{k=1}^{321} 9 \times 10^{k-1}",
        r"(10-1)+(10^2-1)+(10^3-1)+\cdots+(10^{321}-1)": r"\sum_{k=1}^{321} (10^k - 1)",
        r"(10+10^2+10^3+\cdots+10^{321})-321": r"\sum_{k=1}^{321} 10^k - 321",
        r"(10+10^2+10^3+\cdots+10^{321})": r"\sum_{k=1}^{321} 10^k",
        r"1111\cdots.10": r"\frac{10^{322} - 1}{9}",
        r"(10+100+\cdots+10^{320}+10^{321})": r"\sum_{k=0}^{321} 10^k",
        r"11\cdots0": r"\frac{10^{322} - 10}{9}",
        r"11\cdots10-n": r"\frac{10^{322} - 1}{9} - n",
        r"1, 2, 3,\ldots, 19, 20": r"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20",
        r"\dfrac{1}{2} |(a_1b_2 + a_2b_3 + \cdots + a_nb_1) - (b_1a_2 + b_2a_3 + \cdots + b_na_1)|": r"(\sum_{i=1}^{n} a_ib_{i+1} - \sum_{i=1}^{n} b_ia_{i+1})*(1/2)",
        r"(b_1a_2 + b_2a_3 + \cdots + b_na_1)": r"\sum_{i=1}^{n} b_ia_{i+1}",
        r"z_1,z_2,\dots,z_{673}": r"\sum_{i=1}^{673} z_i",
        r"(x-z_1)^3(x-z_2)^3 \cdots (x-z_{673})^3": r"\prod_{i=1}^{673} (x - z_i)^3",
        r"(z_1z_2+z_1z_3+ \dots + z_1z_{672}+z_1z_{673})+(z_2z_3+z_2z_4+ \dots +z_2z_{673}) + (z_3z_4+z_3z_5+ \dots +z_3z_{673}) + \dots +z_{672}z_{673}.": r"\sum_{i=1}^{672} \left( \sum_{j=i+1}^{673} z_iz_j \right)",
        r"P=(x-z_1)(x-z_1)(x-z_1)(x-z_2)(x-z_2)(x-z_2) \dots (x-z_{673})(x-z_{673})(x-z_{673})": r"P = \prod_{i=1}^{673} (x - z_i)^3",
        r"z_1,z_1,z_1,z_2,z_2,z_2, \dots , z_{673},z_{673},z_{673}": r"\sum_{i=1}^{673} \left( z_i + z_i + z_i \right)",
        r"(-1)^1 \cdot \dfrac{20}{1}=-20=z_1+z_1+z_1+z_2+z_2+z_2+ \dots + z_{673}+z_{673}+z_{673}=3(z_1+z_2+z_3+ \dots +z_{673})": r"(-1)^1 \frac{20}{1} = -20 = \sum_{i=1}^{673} 3z_i = 3 \sum_{i=1}^{673} z_i",
        r"z_1+z_2+z_3+ \dots +z_{673}=- \dfrac{20}{3}.": r"\sum_{i=1}^{673} z_i = -\frac{20}{3}",
        r"z_1^2+z_1^2+z_1^2+z_1z_2+z_1z_2+z_1z_2+ \dots =  \\ 3(z_1^2+z_2^2+ \dots + z_{673}^2) + 9(z_1z_2+z_1z_3+z_1z_4+ \dots + z_{672}z_{673}) =  3(z_1^2+z_2^2+ \dots + z_{673}^2) + 9S.": r"\sum_{i=1}^{673} z_i^2 = \frac{400}{9} - 2S",
        r"(z_1+z_2+z_3+ \dots +z_{673})^2= (-20/3)^2=\dfrac{400}{9} \\ =(z_1^2+z_2^2+ \dots + z_{673}^2)+2(z_1z_2+z_1z_3+z_1z_4+ \dots + z_{672}z_{673})=(z_1^2+z_2^2+ \dots + z_{673}^2)+2S.": r"\left( \sum_{i=1}^{673} z_i \right)^2 = \frac{400}{9} - 2S",
        r"z_1+z_2+z_3+ \dots +z_{673}": r"\sum_{i=1}^{673} z_i",
        r"z_1^2+z_2^2+ \dots + z_{673}^2= \dfrac{400}{9} -2S": r"\sum_{i=1}^{673} z_i^2 = \frac{400}{9} - 2S",
        r"f(x)=(x-z_{1})(x-z_{1})\cdots (x-z_{673})": r"f(x) = \prod_{i=1}^{673} (x - z_i)",
        r"(x-z_1)^3(x-z_2)^3(x-z_3)^2\cdots (x-z_{673})^3": r"\prod_{i=1}^{673} (x - z_i)^3",
        r"3(x_1+x_2+\cdots+x_{673})": r"3 \sum_{i=1}^{673} x_i",
        r"3(z_1^2+z_2^2+z_3^2+\dots+z_{673}^2)": r"3 \sum_{i=1}^{673} z_i^2",
        r"-20 = 3(z_1+z_2+z_3+z_4 \dots+z_{673})": r"-20 = 3 \sum_{i=1}^{673} z_i",
        r"19 = 3(z_1^2+z_2^2+z_3^2+\dots+z_{673}^2) + 9S": r"19 = 3 \sum_{i=1}^{673} z_i^2 + 9S",
        r"P_2 = z_1^2+z_2^2+z_3^2+\dots+z_{673}^2": r"P_2 = \sum_{i=1}^{673} z_i^2",
        r"z_1^2+z_2^2+ \dots + z_{673}^2": r"\sum_{i=1}^{673} z_i^2"
    }
    
    # Si la expresión está en el diccionario, devolver la expresión equivalente y almacenarla en to_train_model
    if expression in equivalents:
        interpreted_expression = equivalents[expression]
        return interpreted_expression
    
    # Si no hay una expresión equivalente definida, devolver la misma expresión y no almacenarla en to_train_model
    return expression

In [3]:
def sympy_helper(exp):
    equivalents = {
        r'(c^2 == a^2 + b^2 - 2abcos C)': r'c^2 == a^2 + b^2 - 2ab \cos C',
        r'\angleKLN == \angleKMN == 90^∘': r'\angleKLN == \angleKMN == 90^{\angle}',
        r'm\angleKPL == 90^∘': r'\angleKPL == 90^\angle',
        r'\angleKPL ≅\angleKLN': r'\angleKPL ==\angle',
        r'\anglePKL ≅\angleLKN':r'\anglePKL ==\angleLKN',
        r'PKL ∼LKN':r'\triangle PKL == \triangle LKN',
        r'KMN ∼KPO': r'\triangle KMN == \triangle KPO',
        r'KOL ∼KHP':r'\triangle KOL == \triangle KHP',
        r'd=={090': r'd=={090}',
        r'(a,b)==(20,190).':r'Eq((a, b), (20, 190))',
        r'xy(x, y)^2[x, y]^2==10^630':r'Eq(x * y * (x + y)**2 * x**2 * y**2, 10**630)',
        r'(a, b)*[a, b]==ab':r'Eq(a**2 + b**2, a*b)',
        r'd^3α==10^60, d^3α^2β^3==10^510 αβ^3 == 10^510==2^510 *5^510': [r'Eq(d**3*alpha, 10**60), Eq(d**3*alpha**2*beta**3, 10**510),Eq(alpha*beta**3, 10**510)'],
        r'(a,b,c,d)==(20,20,190,190))))))))':r'Eq((a, b, c, d), (20, 20, 190, 190))',
        r'sin^2x*cos^2x≠=5/6':r'\sin^2 x \cdot \cos^2 x \neq \frac{5}{6}',
        r'a == 1 ±2/3/2':r'[Eq(a,(1 + 2/3) / 2), Eq(a,(1 - 2/3) / 2)]',
        r'p==q^4±1':r'[Eq(p,(q**4 + 1)), Eq(p,(q**4 - 1))]',
        r'| ∑_1 ≤j <k ≤673 z_jz_k |':[r'Abs(Sum(z[j]*z[k], (j, 1, 672), (k, j + 1, 673)))'],
        r'x==∑_1≤j<k≤673 z_jz_k': [r'Eq(x, Sum((z[j] * z[k]), (j, 1, 672), (k, j+1, 673)))'],
        r'3∑_i==1^673z_i==-20∑_i==1^673z_i==-20/3.': r'Eq(3*Sum(z_i, (i, 1, 673)), -20), Eq(Sum(z_i, (i, 1, 673)), -1*20/3)',
        r'1≤j<k≤673': r'1\le j < k\le 673',
        r'| ∑_1 ≤j <k ≤673 z_jz_k |  == S': [r'Eq(S,Abs(Sum(z[j]*z[k], (j, 1, 672), (k, j + 1, 673))))'],
        r'== 3*673+':r'3 * 673'  
    }

    if exp in equivalents:
        interpreted_expression = equivalents[exp]
        return interpreted_expression
    
    return exp

In [4]:
def clean_exp(exp):
    exp = exp.replace('=', '==') 
    exp = exp.replace('pmod', 'mod')
    exp = exp.replace('bmod', 'mod')
    if exp.startswith('$'):
        if re.search(r'\\boxed', exp):
            exp = re.sub(r'\\boxed', '', exp)
        return exp[1:-1]
    if exp.startswith(r'\['):
        if re.search(r'\\boxed', exp):
            exp = re.sub(r'\\boxed', '', exp)
        return exp[2:-2]
    if exp is None:
        print('IS NONE', exp)
    return exp

In [5]:
def generate_context(expr):
    # Identificar las variables simbólicas en la expresión
    symbols = sp.symbols(expr.replace(',', ' ').replace('=', ' ').split())
    context = {str(symbol): symbol for symbol in symbols}
    return context

In [6]:
def parse_expression(expr, context):
    cleaned_expr = clean_exp(expr)
    try:
        return latex2sympy(cleaned_expr)
    except Exception as e:
        try:
            return sp.sympify(cleaned_expr)
        except Exception as e:
            if re.search(r'\\cdots|\\ldots|\\dots', cleaned_expr):
                cleaned_expr = cleaned_expr.replace('==', '=')
                parsed_expr = parse_assigment_with_dots(cleaned_expr)
                parsed_expr = parsed_expr.replace('=', '==')
                try:
                    return latex2sympy(parsed_expr)
                except Exception as e:
                    print('ERROR CDOTS', parsed_expr, e)
            else:
                try:
                    text_expr = LatexNodes2Text().latex_to_text(cleaned_expr)
                    text_expr = text_expr.replace('·', '*')
                    text_expr = text_expr.replace('∠', '\\angle')
                    return latex2sympy(text_expr)
                except Exception as e:
                    try:
                        text_expr = sympy_helper(text_expr)
                        return sp.sympify(text_expr)
                    except Exception as e:
                        try:
                            return latex2sympy(text_expr)
                        except Exception as e:
                            return str(e)
                

In [7]:
def get_variables(parsed_expr):
    try:
        if isinstance(parsed_expr, bool) or parsed_expr is False:
            return []
        if isinstance(parsed_expr, tuple):
            parsed_expr = list(parsed_expr)
        if isinstance(parsed_expr, list):
            variables_list = []
            for item in parsed_expr:
                if isinstance(item, bool) or item is False:
                    continue
                variables_list.extend(item.atoms(sp.Symbol))
            return list(set(variables_list))  # Eliminar duplicados
        variables_list = list(parsed_expr.atoms(sp.Symbol))
        return variables_list
    except Exception as e:
        return str(e)

In [8]:
def process_cell(cell):
    cell_results = []
    cell_var_results = []
    for key, value in cell.items():
        context = generate_context(value)
        parsed = parse_expression(value, context)
        var_colect = get_variables(parsed)
        cell_results.append((key, value, parsed, var_colect))
        cell_var_results.append((key, var_colect))
    return cell_results, cell_var_results

In [9]:

def process_dataframe(dfexpre):
    parsed_results = []
    variables_results = []
    context_variables = set()

    for index, row in dfexpre.iterrows():
        row_results = []
        row_var_results = []
        for cell in row:
            cell_results, cell_var_results = process_cell(cell)
            row_results.append(cell_results)
            row_var_results.append(cell_var_results)
            for _, var_colect in cell_var_results:
                context_variables.update(var_colect)
        parsed_results.append(row_results)
        variables_results.append(row_var_results)

    # Convertir los resultados a un DataFrame plano
    flat_parsed_results = [item for sublist in parsed_results for item in sublist]
    flat_parsed_results = [item for sublist in flat_parsed_results for item in sublist]

    # Crear un DataFrame a partir de los resultados planos
    flat_parsed_df = pd.DataFrame(flat_parsed_results, columns=['Expression', 'Value', 'Parsed', 'Variables'])

    # Contar cuántas expresiones se parsearon correctamente y cuántas no
    flat_parsed_df['Parsed_Success'] = flat_parsed_df['Parsed'].apply(lambda x: not isinstance(x, str))
    parsed_success_count = flat_parsed_df['Parsed_Success'].sum()
    parsed_failure_count = len(flat_parsed_df) - parsed_success_count

    # Mostrar el DataFrame resultante
    
    print(f"Expresiones parseadas correctamente: {parsed_success_count}")
    print(f"Expresiones que no se pudieron parsear: {parsed_failure_count}")

    return parsed_results, variables_results, context_variables, flat_parsed_df

In [10]:
import ast
import pandas as pd
complet_expressions = pd.read_csv('Data/Complet_Expressions.csv', header=None)
dfexpre=pd.DataFrame(complet_expressions)
dfexpre = dfexpre.map(lambda x: ast.literal_eval(x))

In [15]:
parsed_results, variables_results, context_variables, flat_parsed_df= process_dataframe(dfexpre)

      Expression                                              Value  \
0     expresion0  \[N = 9 + 99 + 999 + 9999 + \cdots + \underbra...   
1     expresion1                                                $N$   
2     expresion0                                             $10^n$   
3     expresion1     $(10-1)+(10^2-1)+(10^3-1)+\cdots+(10^{321}-1)$   
4     expresion2               $(10+10^2+10^3+\cdots+10^{321})-321$   
..           ...                                                ...   
894  expresion19  $P_2\cdot a_n + P_1 \cdot a_{n-1} + 2\cdot a_{...   
895  expresion20  \[P_2 + -20\cdot20 + 19\cdot2 = 0 \implies P_2...   
896  expresion21       \[19 = 362 + 9S \implies S = \frac{343}{9}\]   
897  expresion22                                    $\frac{343}{9}$   
898  expresion23                                      $\boxed{352}$   

                                     Parsed  \
0    Eq(N, Sum(9*10**(k - 1), (k, 1, 321)))   
1                                         N   
2     

In [12]:
flat_parsed_df['Parsed'].apply(type).value_counts()

Parsed
<class 'sympy.core.relational.Equality'>             311
<class 'sympy.core.numbers.Integer'>                 140
<class 'sympy.core.symbol.Symbol'>                   133
<class 'sympy.core.mul.Mul'>                         103
<class 'tuple'>                                       49
<class 'list'>                                        39
<class 'sympy.core.add.Add'>                          34
<class 'sympy.core.power.Pow'>                        24
<class 'sympy.core.numbers.One'>                      19
<class 'sympy.concrete.summations.Sum'>                7
<class 'sympy.core.relational.StrictLessThan'>         6
<class 'set'>                                          5
<class 'bool'>                                         4
<class 'function'>                                     3
<class 'sympy.core.relational.LessThan'>               3
<class 'sympy.core.numbers.Zero'>                      2
<class 'str'>                                          2
<class 'sympy.core.relat

In [13]:
parsed_resultsdf = pd.DataFrame(parsed_results)

In [17]:
unique_elements = flat_parsed_df['Parsed'].apply(type).value_counts() == 1
unique_elements_df = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: type(x) in unique_elements[unique_elements].index)]

In [18]:
unique_elements_df.head()

,Expression,Value,Parsed,Variables,Parsed_Success
100,expresion3,$\dbinom{n-k+1}{k}$,"binomial(-k + n + 1, k)","[n, k]",True
134,expresion10,$E$,E,[],True
187,expresion10,$12.5$,25/2,[],True
450,expresion1,"$\gcd(a, b)$",1,[],True
554,expresion18,$\sin^2x\cdot\cos^2x\not=\frac{5}{6}$,"Ne(sin(x*cos(x)**2)**2, 5/6)",[x],True


In [14]:
tuples_df = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: isinstance(x, tuple))]
setdf = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: isinstance(x, set))]
strdf = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: isinstance(x, str))]
listdf = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: isinstance(x, list))]
symboldf = flat_parsed_df[flat_parsed_df['Parsed'].apply(lambda x: isinstance(x, sp.core.symbol.Symbol))]
# Mostrar el DataFrame filtrado
symboldf.head()


,Expression,Value,Parsed,Variables,Parsed_Success
1,expresion1,$N$,N,[N],True
32,expresion3,$n$,n,[n],True
42,expresion6,$N$,N,[N],True
44,expresion0,$J$,J,[J],True
46,expresion2,$B$,B,[B],True


In [59]:
reference_text = pd.read_csv('Data/refefenceText.csv', header=None)
dfrefrence=pd.DataFrame(reference_text)


In [ ]:
def extract_parsed_expressions(dfparsedresults):
    parsed_list=[]
    for i, col in enumerate(dfparsedresults):
        parsed_list_row=[]
        for j,row in enumerate(dfparsedresults[i]):
            parsed_dict = {}    
            for exp in row:
                if isinstance(exp, tuple) and len(exp) >= 3:
                    expr_key = exp[0]
                    parsed_expr = exp[2]
                    parsed_dict[expr_key] = parsed_expr
                else:
                    print(len(exp), exp)
            parsed_list_row.append(parsed_dict)
        parsed_list.append(parsed_list_row)
    return parsed_list
            
parsed_list = extract_parsed_expressions(dfparsedresults) 

In [63]:
# Función para convertir una cadena de caracteres en una frase completa
def convert_string_to_sentence(text):
    if isinstance(text, str):
        words = eval(text)  # Convertir la cadena de caracteres en una lista de palabras
        return ' '.join(words)  # Unir las palabras en una frase completa
    else:
        return text

# Aplicar la función a cada celda de la columna que contenga texto en lenguaje natural en dfrefrence
for col in dfrefrence.columns:
    if dfrefrence[col].dtype == 'object':  # Asegurarse de que la columna contenga texto
        dfrefrence[col] = dfrefrence[col].apply(convert_string_to_sentence)

# Verificar el resultado
print(dfrefrence.head())

                                                   0  \
0  Consider the integer expresion0 Find the sum o...   
1  Jenn randomly chooses a number expresion0 from...   
2  In expresion0 expresion1 expresion2 and expres...   
3  A soccer team has expresion0 available players...   
4  A moving particle starts at the point expresio...   

                                                   1  \
0  Let s express the number in terms of expresion...   
1  By symmetry the desired probability is equal t...   
2  We know the area of the hexagon expresion0 to ...   
3  There are expresion0 substitutions The number ...   
4  One could recursively compute the probabilitie...   

                                                   2  \
0  A similar and simpler way to consider the init...   
1  By symmetry the desired probability is equal t...   
2  Let expresion0 be the origin Noticing that the...   
3  There is expresion0 way of making no substitut...   
4  Obviously the only way to reach expresion0 

In [64]:
dfrefrence[0][0]

'Consider the integer expresion0 Find the sum of the digits of expresion1'

In [65]:
import spacy

# Cargar el modelo de spaCy para inglés
nlp = spacy.load("en_core_web_sm")

# Función para tokenizar y lematizar
def tokenize_and_lemmatize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_punct and not token.is_space]

# Crear un DataFrame intermedio para tokenización y lematización
tokenizerdf = pd.DataFrame()

# Aplicar la función a cada columna que contenga texto en lenguaje natural
for col in dfrefrence.columns:
    if dfrefrence[col].dtype == 'object':
        tokenizerdf[f'{col}_Tokens'] = dfrefrence[col].apply(tokenize_and_lemmatize)

# Verificar el resultado
print(tokenizerdf.head())

                                            0_Tokens  \
0  [consider, the, integer, expresion0, find, the...   
1  [Jenn, randomly, choose, a, number, expresion0...   
2  [in, expresion0, expresion1, expresion2, and, ...   
3  [a, soccer, team, have, expresion0, available,...   
4  [a, move, particle, start, at, the, point, exp...   

                                            1_Tokens  \
0  [let, s, express, the, number, in, term, of, e...   
1  [by, symmetry, the, desire, probability, be, e...   
2  [we, know, the, area, of, the, hexagon, expres...   
3  [there, be, expresion0, substitution, the, num...   
4  [one, could, recursively, compute, the, probab...   

                                            2_Tokens  \
0  [a, similar, and, simple, way, to, consider, t...   
1  [by, symmetry, the, desire, probability, be, e...   
2  [let, expresion0, be, the, origin, Noticing, t...   
3  [there, be, expresion0, way, of, make, no, sub...   
4  [obviously, the, only, way, to, reach, expr

In [67]:
import gensim.downloader as api

# Cargar modelos preentrenados
word2vec_model = api.load("word2vec-google-news-300")
glove_model = api.load("glove-wiki-gigaword-300")

# Función para obtener embeddings de una lista de tokens
def get_embeddings(tokens, model):
    embeddings = [model[token] for token in tokens if token in model]
    return embeddings



[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 376.1/376.1MB downloaded


AttributeError: 'int' object has no attribute 'endswith'

In [69]:
# Aplicar la función a cada columna lematizada en dfrefrence
for col in tokenizerdf.columns:
    if col.endswith('Tokens'):
        tokenizerdf[f'{col}_Word2Vec_Embeddings'] = tokenizerdf[col].apply(lambda x: get_embeddings(x, word2vec_model))
        tokenizerdf[f'{col}_GloVe_Embeddings'] = tokenizerdf[col].apply(lambda x: get_embeddings(x, glove_model))


In [70]:
tokenizerdf.head()

,0_Tokens,1_Tokens,2_Tokens,3_Tokens,4_Tokens,5_Tokens,0_Tokens_Word2Vec_Embeddings,0_Tokens_GloVe_Embeddings,1_Tokens_Word2Vec_Embeddings,1_Tokens_GloVe_Embeddings,2_Tokens_Word2Vec_Embeddings,2_Tokens_GloVe_Embeddings,3_Tokens_Word2Vec_Embeddings,3_Tokens_GloVe_Embeddings,4_Tokens_Word2Vec_Embeddings,4_Tokens_GloVe_Embeddings,5_Tokens_Word2Vec_Embeddings,5_Tokens_GloVe_Embeddings
0,"[consider, the, integer, expresion0, find, the...","[let, s, express, the, number, in, term, of, e...","[a, similar, and, simple, way, to, consider, t...","[we, can, see, that, expresion0, expresion1, e...","[observe, how, add, result, in, the, last, ter...","[write, N, expresion0, expresion1, expresion2,...","[[-0.06347656, 0.09082031, 0.1953125, 0.093261...","[[0.38731, 0.053821, 0.29723, 0.048756, 0.1740...","[[0.20800781, 0.039794922, 0.25, 0.24414062, -...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....","[[0.05883789, 0.06201172, 0.07080078, 0.159179...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...","[[-0.11035156, -0.013916016, 0.11376953, 0.177...","[[0.25236, 0.17171, -0.27513, -0.22323, -0.644...","[[-0.03930664, -0.18164062, 0.037597656, -0.04...","[[-0.32122, 0.024329, -0.49585, -0.41928, -0.1..."
1,"[Jenn, randomly, choose, a, number, expresion0...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[this, problem, be, essentially, ask, how, man...","[create, a, grid, use, graph, paper, with, exp...","[[-0.025512695, -0.042236328, -0.23632812, -0....","[[-0.68857, 1.0471, 0.39247, 0.62252, 0.047625...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...","[[0.109375, 0.140625, -0.03173828, 0.16601562,...","[[-0.20437, 0.16431, 0.041794, -0.13708, -0.29...","[[-0.03857422, 0.020629883, -0.03149414, -0.05...","[[-0.36201, 0.26625, -0.498, -0.29232, 0.01494..."
2,"[in, expresion0, expresion1, expresion2, and, ...","[we, know, the, area, of, the, hexagon, expres...","[let, expresion0, be, the, origin, Noticing, t...","[note, that, expresion0, have, area, expresion...","[know, that, expresion0, have, area, expresion...","[triangle, expresion0, be, a, right, triangle,...","[[0.0703125, 0.08691406, 0.087890625, 0.0625, ...","[[-0.44399, 0.12817, -0.25247, -0.18582, -0.16...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...","[[0.20800781, 0.039794922, 0.25, 0.24414062, -...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....","[[-0.087402344, 0.25390625, -0.21777344, -0.13...","[[-0.12192, -0.0082955, -0.23558, -0.16759, -0...","[[-0.05493164, -0.1171875, 0.027832031, 0.0747...","[[-0.21054, 0.1382, 0.035328, 0.03977, -0.1091...","[[-0.1640625, -0.071777344, -0.036865234, 0.07...","[[0.11336, -0.26752, 0.12971, -0.53841, -0.154..."
3,"[a, soccer, team, have, expresion0, available,...","[there, be, expresion0, substitution, the, num...","[there, be, expresion0, way, of, make, no, sub...","[we, can, perform, casework, call, the, substi...","[no, more, solution, explain]","[no, more, solution, explain]","[[-0.076660156, 0.11035156, 0.35351562, -0.079...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[0.09423828, -0.022827148, 0.052246094, 0.026...","[[-0.47618, 0.069478, -0.013086, 0.1742, -0.31...","[[0.09423828, -0.022827148, 0.052246094, 0.026...","[[-0.47618, 0.069478, -0.013086, 0.1742, -0.31...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...","[[0.08251953, -0.15136719, 0.06591797, 0.02001...","[[-0.16843, -0.037651, -0.17304, -0.069757, -0...","[[0.08

In [71]:
from transformers import BertTokenizer, BertModel
import torch

# Cargar el tokenizador y el modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')



A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [73]:
# Función para obtener embeddings BERT de una lista de tokens
def get_bert_embeddings(tokens):
    inputs = tokenizer(tokens, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Promedio de los embeddings de los tokens
    return embeddings.numpy()

# Aplicar la función a cada columna lematizada en dfrefrence
for col in tokenizerdf.columns:
    if col.endswith('Tokens'):
        print('... in this column', col)
        tokenizerdf[f'{col}_BERT_Embeddings'] = tokenizerdf[col].apply(lambda x: get_bert_embeddings(x))

... in this column 0_Tokens
... in this column 1_Tokens
... in this column 2_Tokens
... in this column 3_Tokens
... in this column 4_Tokens
... in this column 5_Tokens
                                                   0  \
0  Consider the integer expresion0 Find the sum o...   
1  Jenn randomly chooses a number expresion0 from...   
2  In expresion0 expresion1 expresion2 and expres...   
3  A soccer team has expresion0 available players...   
4  A moving particle starts at the point expresio...   

                                                   1  \
0  Let s express the number in terms of expresion...   
1  By symmetry the desired probability is equal t...   
2  We know the area of the hexagon expresion0 to ...   
3  There are expresion0 substitutions The number ...   
4  One could recursively compute the probabilitie...   

                                                   2  \
0  A similar and simpler way to consider the init...   
1  By symmetry the desired probability is equa

In [74]:
tokenizerdf.head()

,0_Tokens,1_Tokens,2_Tokens,3_Tokens,4_Tokens,5_Tokens,0_Tokens_Word2Vec_Embeddings,0_Tokens_GloVe_Embeddings,1_Tokens_Word2Vec_Embeddings,1_Tokens_GloVe_Embeddings,...,4_Tokens_Word2Vec_Embeddings,4_Tokens_GloVe_Embeddings,5_Tokens_Word2Vec_Embeddings,5_Tokens_GloVe_Embeddings,0_Tokens_BERT_Embeddings,1_Tokens_BERT_Embeddings,2_Tokens_BERT_Embeddings,3_Tokens_BERT_Embeddings,4_Tokens_BERT_Embeddings,5_Tokens_BERT_Embeddings
0,"[consider, the, integer, expresion0, find, the...","[let, s, express, the, number, in, term, of, e...","[a, similar, and, simple, way, to, consider, t...","[we, can, see, that, expresion0, expresion1, e...","[observe, how, add, result, in, the, last, ter...","[write, N, expresion0, expresion1, expresion2,...","[[-0.06347656, 0.09082031, 0.1953125, 0.093261...","[[0.38731, 0.053821, 0.29723, 0.048756, 0.1740...","[[0.20800781, 0.039794922, 0.25, 0.24414062, -...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....",...,"[[-0.11035156, -0.013916016, 0.11376953, 0.177...","[[0.25236, 0.17171, -0.27513, -0.22323, -0.644...","[[-0.03930664, -0.18164062, 0.037597656, -0.04...","[[-0.32122, 0.024329, -0.49585, -0.41928, -0.1...","[[0.083200805, 0.08995607, -0.25757357, -0.188...","[[0.122171395, -0.22871114, 0.21894948, 0.0019...","[[-0.32096776, -0.4020112, 0.21519597, 0.01935...","[[0.56759626, -0.28349862, 0.096860595, 0.0276...","[[0.02203683, 0.21115059, 0.03888317, 0.110697...","[[-0.0063941604, 0.14518824, 0.16095427, -0.27..."
1,"[Jenn, randomly, choose, a, number, expresion0...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[this, problem, be, essentially, ask, how, man...","[create, a, grid, use, graph, paper, with, exp...","[[-0.025512695, -0.042236328, -0.23632812, -0....","[[-0.68857, 1.0471, 0.39247, 0.62252, 0.047625...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...",...,"[[0.109375, 0.140625, -0.03173828, 0.16601562,...","[[-0.20437, 0.16431, 0.041794, -0.13708, -0.29...","[[-0.03857422, 0.020629883, -0.03149414, -0.05...","[[-0.36201, 0.26625, -0.498, -0.29232, 0.01494...","[[0.12856832, -0.28871277, 0.056057554, 0.0198...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.08260063, -0.16870154, 0.068083465, 0.118...","[[0.21397035, -0.075880595, -0.23536241, 0.144..."
2,"[in, expresion0, expresion1, expresion2, and, ...","[we, know, the, area, of, the, hexagon, expres...","[let, expresion0, be, the, origin, Noticing, t...","[note, that, expresion0, have, area, expresion...","[know, that, expresion0, have, area, expresion...","[triangle, expresion0, be, a, right, triangle,...","[[0.0703125, 0.08691406, 0.087890625, 0.0625, ...","[[-0.44399, 0.12817, -0.25247, -0.18582, -0.16...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...",...,"[[-0.05493164, -0.1171875, 0.027832031, 0.0747...","[[-0.21054, 0.1382, 0.035328, 0.03977, -0.1091...","[[-0.1640625, -0.071777344, -0.036865234, 0.07...","[[0.11336, -0.26752, 0.12971, -0.53841, -0.154...","[[-0.3334507, -0.34485006, -0.16712357, 0.0266...","[[0.56759626, -0.28349862, 0.096860595, 0.0276...","[[0.122171395, -0.22871114, 0.21894948, 0.0019...","[[-0.10713527, 0.25711396, 0.102142654, -0.003...","[[0.05898134, -0.023897639, 0.08185868, 0.0844...","[[-0.05892214, -0.04398841, -0.1415897, 0.0601..."
3,"[a, soccer, team, have, expresion0, available,...","[there, be, expresion0, substitution, the, num...","[there, be, expresion0, way, of, make, no, sub...","[we, can, perform, casework, call, the, substi...","[no, more, solution, explain]","[no, more, solution, explain]","[[-0.076660156, 0.11035156, 0.35351562, -0.079...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[0.09423828, -0.022827148, 0.052246094, 0.026...","[[

In [79]:
dfparsedresults = pd.DataFrame(parsed_results)
dfparsedresults[0][1]

[('expresion0', '$J$', J, [J]),
 ('expresion1',
  '$1, 2, 3,\\ldots, 19, 20$',
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
  []),
 ('expresion2', '$B$', B, [B]),
 ('expresion3',
  '$1, 2, 3,\\ldots, 19, 20$',
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
  []),
 ('expresion4', '$J$', J, [J]),
 ('expresion5', '$B - J$', B - J, [J, B]),
 ('expresion6', '$2$', 2, []),
 ('expresion7', '$\\frac{m}{n}$', m/n, [n, m]),
 ('expresion8', '$m$', m, [m]),
 ('expresion9', '$n$', n, [n]),
 ('expresion10', '$m+n$', m + n, [n, m])]

In [89]:
import pandas as pd

# Supongamos que dfparsedresults es un DataFrame de pandas
print(type(dfparsedresults))  # Debería imprimir <class 'pandas.core.frame.DataFrame'>

# Verificar las primeras filas del DataFrame
print(dfparsedresults.head())

# Verificar la estructura de una fila específica
print(dfparsedresults.iloc[0])

<class 'pandas.core.frame.DataFrame'>
                                                   0  \
0  [(expresion0, \[N = 9 + 99 + 999 + 9999 + \cdo...   
1  [(expresion0, $J$, J, [J]), (expresion1, $1, 2...   
2  [(expresion0, $\triangle PQR$, P*Q*R*\triangle...   
3  [(expresion0, $22$, 22, []), (expresion1, $11$...   
4  [(expresion0, $(4,4)$, (4, 4), []), (expresion...   

                                                   1  \
0  [(expresion0, $10^n$, 10**n, [n]), (expresion1...   
1  [(expresion0, $J - B$, -B + J, [J, B]), (expre...   
2  [(expresion0, $ABCDEF$, A*B*C*D, [A, B, D, C])...   
3  [(expresion0, $0-3$, -3 + 0, []), (expresion1,...   
4  [(expresion0, $(0,0)$, (0, 0), []), (expresion...   

                                                   2  \
0  [(expresion0, $1$, 1, []), (expresion1, $(10+1...   
1  [(expresion0, $J - B$, -B + J, [J, B]), (expre...   
2  [(expresion0, $R$, R, [R]), (expresion1, $A=(4...   
3  [(expresion0, $1$, 1, []), (expresion1, $11^2$...   
4  [(exp

In [90]:
def extract_parsed_expressions(dfparsedresults):
    parsed_dict = {}
    for index, row in dfparsedresults.iterrows():
        for cell in row:
            if isinstance(cell, list):
                for expr_tuple in cell:
                    if isinstance(expr_tuple, tuple) and len(expr_tuple) >= 3:
                        expr_key = expr_tuple[0]
                        parsed_expr = expr_tuple[2]
                        parsed_dict[expr_key] = parsed_expr
    return parsed_dict

In [188]:


# Verificar el resultado
print(parsed_list[0][1])

{'expresion0': J, 'expresion1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'expresion2': B, 'expresion3': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'expresion4': J, 'expresion5': B - J, 'expresion6': 2, 'expresion7': m/n, 'expresion8': m, 'expresion9': n, 'expresion10': m + n}


In [160]:
parsed_results[0][0]

[('expresion0',
  '\\[N = 9 + 99 + 999 + 9999 + \\cdots + \\underbrace{99\\ldots 99}_\\text{321 digits}.\\]',
  Eq(N, Sum(9*10**(k - 1), (k, 1, 321))),
  [k, N]),
 ('expresion1', '$N$', N, [N])]

In [193]:
def replace_expressions(text, parsed_list):
    for key, value in parsed_list.items():
        text = text.replace(key, str(value))
    return text


for i, col in enumerate(tokenizerdf):
    if col.endswith('Tokens'):
        for j,row in enumerate(tokenizerdf[col]):
            new_row=[]
            for word in row:
                if word.startswith('expresion'):
                    parsed= replace_expressions(word, parsed_list[i][j])
                    new_row.append(parsed)
                else:
                    new_row.append(word)
            tokenizerdf[col].iloc[j] = new_row


FORMA 240
ENTRANDO EN COLUMNA 0 0_Tokens
ENTRANDO EN FILA 0 ['consider', 'the', 'integer', 'expresion0', 'find', 'the', 'sum', 'of', 'the', 'digit', 'of', 'expresion1']
ENTRANDO EN FILA 1 ['Jenn', 'randomly', 'choose', 'a', 'number', 'expresion0', 'from', 'expresion1', 'Bela', 'then', 'randomly', 'choose', 'a', 'number', 'expresion2', 'from', 'expresion3', 'distinct', 'from', 'expresion4', 'the', 'value', 'of', 'expresion5', 'be', 'at', 'least', 'expresion6', 'with', 'a', 'probability', 'that', 'can', 'be', 'express', 'in', 'the', 'form', 'expresion7', 'where', 'expresion8', 'and', 'expresion9', 'be', 'relatively', 'prime', 'positive', 'integer', 'find', 'expresion10']
ENTRANDO EN FILA 2 ['in', 'expresion0', 'expresion1', 'expresion2', 'and', 'expresion3', 'Points', 'expresion4', 'and', 'expresion5', 'lie', 'on', 'expresion6', 'point', 'expresion7', 'and', 'expresion8', 'lie', 'on', 'expresion9', 'and', 'point', 'expresion10', 'and', 'expresion11', 'lie', 'on', 'expresion12', 'with', '

In [194]:
tokenizerdf.head()

,0_Tokens,1_Tokens,2_Tokens,3_Tokens,4_Tokens,5_Tokens,0_Tokens_Word2Vec_Embeddings,0_Tokens_GloVe_Embeddings,1_Tokens_Word2Vec_Embeddings,1_Tokens_GloVe_Embeddings,...,4_Tokens_Word2Vec_Embeddings,4_Tokens_GloVe_Embeddings,5_Tokens_Word2Vec_Embeddings,5_Tokens_GloVe_Embeddings,0_Tokens_BERT_Embeddings,1_Tokens_BERT_Embeddings,2_Tokens_BERT_Embeddings,3_Tokens_BERT_Embeddings,4_Tokens_BERT_Embeddings,5_Tokens_BERT_Embeddings
0,"[consider, the, integer, Eq(N, Sum(9*10**(k - ...","[let, s, express, the, number, in, term, of, 1...","[a, similar, and, simple, way, to, consider, t...","[we, can, see, that, Eq(9, 9), Eq(9 + 99, 108)...","[observe, how, add, result, in, the, last, ter...","[write, N, -1 + 10, -1 + 10**2, -1 + 10**321, ...","[[-0.06347656, 0.09082031, 0.1953125, 0.093261...","[[0.38731, 0.053821, 0.29723, 0.048756, 0.1740...","[[0.20800781, 0.039794922, 0.25, 0.24414062, -...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....",...,"[[-0.11035156, -0.013916016, 0.11376953, 0.177...","[[0.25236, 0.17171, -0.27513, -0.22323, -0.644...","[[-0.03930664, -0.18164062, 0.037597656, -0.04...","[[-0.32122, 0.024329, -0.49585, -0.41928, -0.1...","[[0.083200805, 0.08995607, -0.25757357, -0.188...","[[0.122171395, -0.22871114, 0.21894948, 0.0019...","[[-0.32096776, -0.4020112, 0.21519597, 0.01935...","[[0.56759626, -0.28349862, 0.096860595, 0.0276...","[[0.02203683, 0.21115059, 0.03888317, 0.110697...","[[-0.0063941604, 0.14518824, 0.16095427, -0.27..."
1,"[Jenn, randomly, choose, a, number, J, from, [...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[by, symmetry, the, desire, probability, be, e...","[this, problem, be, essentially, ask, how, man...","[create, a, grid, use, graph, paper, with, 20,...","[[-0.025512695, -0.042236328, -0.23632812, -0....","[[-0.68857, 1.0471, 0.39247, 0.62252, 0.047625...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...",...,"[[0.109375, 0.140625, -0.03173828, 0.16601562,...","[[-0.20437, 0.16431, 0.041794, -0.13708, -0.29...","[[-0.03857422, 0.020629883, -0.03149414, -0.05...","[[-0.36201, 0.26625, -0.498, -0.29232, 0.01494...","[[0.12856832, -0.28871277, 0.056057554, 0.0198...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.15728115, -0.111764036, 0.40003172, 0.231...","[[-0.08260063, -0.16870154, 0.068083465, 0.118...","[[0.21397035, -0.075880595, -0.23536241, 0.144..."
2,"[in, P*Q*R*\triangle, Eq(P*R, 15), Eq(Q*R, 20)...","[we, know, the, area, of, the, hexagon, A*B*C*...","[let, R, be, the, origin, Noticing, that, the,...","[note, that, P*Q*R*\triangle, have, area, 150,...","[know, that, P*Q*R*\triangle, have, area, 150,...","[triangle, P*Q*R, be, a, right, triangle, with...","[[0.0703125, 0.08691406, 0.087890625, 0.0625, ...","[[-0.44399, 0.12817, -0.25247, -0.18582, -0.16...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...",...,"[[-0.05493164, -0.1171875, 0.027832031, 0.0747...","[[-0.21054, 0.1382, 0.035328, 0.03977, -0.1091...","[[-0.1640625, -0.071777344, -0.036865234, 0.07...","[[0.11336, -0.26752, 0.12971, -0.53841, -0.154...","[[-0.3334507, -0.34485006, -0.16712357, 0.0266...","[[0.56759626, -0.28349862, 0.096860595, 0.0276...","[[0.122171395, -0.22871114, 0.21894948, 0.0019...","[[-0.10713527, 0.25711396, 0.102142654, -0.003...","[[0.05898134, -0.023897639, 0.08185868, 0.0844...","[[-0.05892214, -0.04398841, -0.1415897, 0.0601..."
3,"[a, soccer, team, have, 22, available, player,...","[there, be, -3 + 0, substitution, the, number,...","[there, be, 1, way, of, make, no, substitution...","[we, can, perform, casework, call, the, substi...","[no, more, solution, explain]","[no, more, solution, explain]","[[-0.076660156, 0.11035156, 0.35351562, -0.079...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[0.09423828, -0.022827148, 0.052246094, 0.026...","[[